# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [20]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import modules.auxiliary as aux
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Ceteris paribus una opción sobre una activo con reversión a la media vale menos
que una opción sobre un activo sin reversión a la media. Comente.


Pregunta 2:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después).


Pregunta 3:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 982 y 1454 días.


Pregunta 4:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 5:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .1.4756 .


Pregunta 6:

Considere un swap a 1Y con cupones semestrales de 1.4% (lineal). Si la tasa
cupón cero a 0.5Y es 1.3% (exp), calcule la tasa cero a 1Y.


Pregunta 7:

Usando la curva cero cu

## Respuestas

### Pregunta 1

Ceteris paribus una opción sobre una activo con reversión a la media vale menos
que una opción sobre un activo sin reversión a la media. Comente.

**R: Una opción que presenta reversión a la media tiene un mayor precio debido a que la distribución
de la opción es mucho más estable, lo que permite una mejor predicción del comportamiento futuro, por lo que
no tiende a "escaparse" sus valores, es decir, en el largo plazo se sabe que tiende a converger a una media. Sin embargo, aquel instrumento financiero que no presenta reversión a la media puede tener distribución menos estable y
podría el precio escaparse de la media.**

### Pregunta 2

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después).

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

In [17]:
ICP_365 = 10049.0
ICP_0 = 10000.0
d = 365
TNA = round((ICP_365/ICP_0 -1)*(360/d),4)
print(f'Resultado TNA:{TNA:f}')

Resultado TNA:0.004800


### Pregunta 3

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 982 y 1454 días.

In [18]:
#Interpolación
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

In [28]:
t1 = 982
t2 = 1454
r1 = curva(t1)
r2 = curva(t2)
df_982 = math.exp(-r1 * t1 / 365)
df_1454 = math.exp(-r2 * t2 / 365)
fwd_rate = (df_982 / df_1454 - 1.0) * 360.0 / (t2 - t1)
print(f'La tasa forward entre 982 días y 1454 días es de {fwd_rate:.8%}')


La tasa forward entre 982 días y 1454 días es de 0.23687660%


### Pregunta 4

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.

**R: El valorizar en la medida libre de riesgo implica que uno descuento los flujos del payoff de la opción
con los factores de descuento que simuló con el modelo, por lo tanto, se utiliza el mismo camino que
sigue la tasa para descontar y traer a valor presente el payoff. Sin embargo, el utilizar la medida forward
uno utiliza el factor de descuento de la curva cero cupon al plazo que corresponde el vencimiento, por lo tanto,
este payoff se descuenta con la tasa esperada de la curva**

### Pregunta 5

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .1.4756 .

In [39]:
gamma = 1
sigma = 0.005
dt = 1/264
r0 = curva(0)
r_ = 0.005
alea = 1.4756
drt = gamma*(r_ - r0)*dt + sigma*math.sqrt(dt)*alea
r1 = r0 + drt
print(f'El resultado tras una simulación es:{r1:.8%}')

El resultado tras una simulación es:0.12761808%


### Pregunta 6

Considere un swap a 1Y con cupones semestrales de 1.4% (lineal). Si la tasa
cupón cero a 0.5Y es 1.3% (exp), calcule la tasa cero a 1Y.

In [73]:
wf1 = 1 + 0.014*0.5
wf2 = math.exp(0.013)**(1/2)
tasa = ((wf1*wf2) -1)*1
print(f'Tasa cero 1Y: {tasa:.8%}')

Tasa cero 1Y: 1.35668190%


### Pregunta 7

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.5%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.

In [44]:
from scipy.stats import norm

In [45]:
def option_price(strike,r,gamma,sigma,zrate,fwd,t,Tb,To,e):
    """
    Función calcula el precio de una call o put sobre bonos cupón cero con el modelo de HW. Se tomará como fecha
    de vencimiento 1Y.
    """
    z_tb = zero_hw(r, gamma, sigma, zrate, fwd, t, Tb)
    z_to = zero_hw(r, gamma, sigma, zrate, fwd, t, To)
    d1 = get_d1(z_tb,z_to,strike,gamma,Tb,To)
    d2 = get_d2(z_tb,z_to,strike,gamma,Tb,To)
    
    price = z_to*(e*(z_tb/z_to)*norm.cdf(e*d1) - e*strike*norm.cdf(e*d2))
        
    return price

In [46]:
def get_d1(z_tb,z_to,strike,gamma,Tb,To):
    
    """
    Calcula la funcion d1 de formula de opciones
    """

    sz = s_z(gamma,sigma,Tb,To)
    aux1 = math.log(z_tb/(strike*z_to))
    d1 = (aux1 + 0.5*sz**2)/sz
    return d1

In [47]:
def get_d2(z_tb,z_to,strike,gamma,Tb,To):
    
    """
    Calcula la funcion d2 de formula de opciones
    """
    
    d1 = get_d1(z_tb,z_to,strike,gamma,Tb,To)
    sz = s_z(gamma,sigma,Tb,To)
    d2 = d1 - sz
    return d2

In [58]:
def s_z(gamma:float,
        sigma:float,
        Tb: float,To: float) -> float:
    
    """
    Calcula la funcion Sz
    """
    
    B = b_hw(gamma,To,Tb)
    aux1 = 1 - math.exp(-2*gamma*To)
    aux2 = (sigma**2)/(2*gamma)
    sz = B*math.sqrt(aux2*aux1)
    return sz

In [55]:
def zrate(t: float) -> float:
    return curva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (curva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [56]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [51]:
def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [52]:
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [53]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [63]:
sigma = .005
gamma = 1
r0 = curva(0)
t = 0
To = 1
Tb = 2
strike = 0.985

price_call = option_price(strike,r0,gamma,sigma,zrate,fwd,t,Tb,To,1)
print("Tiempo vencimiento bono es",Tb,"años y tiempo vencimiento de la opcion es",To,"año")
print(f'Precio Call: {price_call:.8f}')

Tiempo vencimiento bono es 2 años y tiempo vencimiento de la opcion es 1 año
Precio Call: 0.01416793
